**Import libraries**

In [ ]:
!pip install https://github.com/ElsevierDev/elsapy/archive/master.zip

In [ ]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [ ]:
#Import some basic libraries
import numpy as np
import pandas as pd
from pprint import pprint

In [ ]:
#Some important elsevier api libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import csv
import requests
import xmltodict
import urllib3
import re

#Import warnings library
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#config = {'apikey': '22d18e0c4fdc59bb8898844056b2be57'}
config = {
    "apikey": "Enter your API key",
 "insttoken": "Enter your institute token"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

### Science Direct Full Text API

In [ ]:
#The data loaded here with links
df = pd.read_csv(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\doi_6000_withlink.csv')
df.head()

**Random check the links** 

In [ ]:
#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": config['apikey'], "X-ELS-Insttoken": config['insttoken'], "Accept": "application/xml"}

yes = 0
no = 0
for i in range(0, df.shape[0]):
    #x takes response of the HTTP request, passes link
    x = requests.get(df['Link'][i], headers=headers_dict)
    #print(x.text) #check
        
    #saving it as XML file
    with open("full_text.xml", 'wb') as f:
        f.write(x.content)
        
    tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
    root = tree.getroot()
    
    original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
    doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
    serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
    print(serial_item)

    if serial_item != None:
        yes +=1
        print(yes)
    else:
        #print("Full text for this paper doesn't exist")
        no+=1
        
print(yes)
print(no)

### Code to Extract Coredata

In [ ]:
#Coredata is present in every paper

#Element.findall() finds only elements with a tag which are direct children of the current element
#Element.find() finds the first child with a particular tag

#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": config['apikey'], "X-ELS-Insttoken": config['insttoken'], "Accept": "application/xml"}

#x takes response of the HTTP request, random passes link
x = requests.get(df['Link'][3], headers=headers_dict)
#print(x.text) #check
    
#saving it as XML file
with open("full_text.xml", 'wb') as f:
    f.write(x.content)

#Reading the data
#<?xml version="1.0" encoding="UTF-8"?>
tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
root = tree.getroot()

for entry in root.findall('{http://www.elsevier.com/xml/svapi/article/dtd}coredata'):
    
    url = []; doi =[]; title = []; pub_name =[]; type_ = []
    description = []
    #CHECK CODE
    url.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text)
    doi.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text)
    title.append(entry.find('{http://purl.org/dc/elements/1.1/}title').text)
    pub_name.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text)
    type_.append(entry.find('{http://prismstandard.org/namespaces/basic/2.0/}aggregationType').text)
    description.append(entry.find('{http://purl.org/dc/elements/1.1/}description').text)
    description[0] = " ".join(description[0].split())
    print('URL:',url )
    print('DOI:', doi)
    print('Title:', title)
    print('Journal name:',pub_name )
    print('Type:',type_ )
    print('\n')
    print('Abstract:',description[0] )
    print('\n')
    
#Extract the list of authors and keywords mentioned      
original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
#print(originaltext)
doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
#print(doc)
serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')
#print(serial_item)

keyword_list = []
author_list = []
if serial_item != None:
    article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
    #print(article)
    head = article.find('{http://www.elsevier.com/xml/ja/dtd}head')
    #print(head)
    author_group = head.find('{http://www.elsevier.com/xml/common/dtd}author-group')
    #print(author_group)

    keywords_ = head.find('{http://www.elsevier.com/xml/common/dtd}keywords')
    #print(keywords_)

    for author in author_group.findall('{http://www.elsevier.com/xml/common/dtd}author'):
        name = author.find('{http://www.elsevier.com/xml/common/dtd}given-name').text
        surname = author.find('{http://www.elsevier.com/xml/common/dtd}surname').text
        author_list.append(name + ' ' + surname)
        #print('\n')

    for word in keywords_.itertext():
        keyword_list.append(word)

    keyword_list = "".join(keyword_list)
    keyword_list = list(keyword_list.split())
    #keywords, authors present in the paper
    print('Keywords present:',keyword_list)
    print('Authors: ',author_list)
else:
    print("Full text for this paper doesn't exist")

### Code to extract full text and store in csv

In [ ]:
#defining a header's dictionary to pass through requests
headers_dict = {"X-ELS-APIKey": "Enter your API key", "X-ELS-Insttoken": "Enter your institute token", "Accept": "application/xml"}
df_temp = pd.DataFrame()
excel_writer = pd.ExcelWriter(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\test2.xlsx', engine='xlsxwriter')
for i in np.arange(0,2,1):
    print(i, 'done' )
    #x takes response of the HTTP request, passes link
    x = requests.get(df['Link'][i], headers=headers_dict)

    #saving it as XML file
    with open("full_text.xml", 'wb') as f:
        f.write(x.content)

    #Reading the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\full_text.xml')
    root = tree.getroot()
    original_text = root.find('{http://www.elsevier.com/xml/svapi/article/dtd}originalText')
    
    if original_text != None:
        doc = original_text.find('{http://www.elsevier.com/xml/xocs/dtd}doc')
        serial_item = doc.find('{http://www.elsevier.com/xml/xocs/dtd}serial-item')

        if serial_item != None:
            article = serial_item.find('{http://www.elsevier.com/xml/ja/dtd}article')
            
            if article != None:
                body = article.find('{http://www.elsevier.com/xml/ja/dtd}body')
                
                if body != None:
                    sections = body.find('{http://www.elsevier.com/xml/common/dtd}sections')
                    
                    #lists for dataframe
                    label_list = [] 
                    title_list = []
                    para_list = []

                    for section1 in sections.findall('{http://www.elsevier.com/xml/common/dtd}section'):
                        if section1.find('{http://www.elsevier.com/xml/common/dtd}label') != None:
                        #if section1 != None:
                            label = section1.find('{http://www.elsevier.com/xml/common/dtd}label').text
                            label_list.append(label)
                            section_title_list = []
                            section_title = section1.find('{http://www.elsevier.com/xml/common/dtd}section-title')
                            for sec in section_title.itertext():
                                section_title_list.append(sec)

                            section_title_list = "".join(section_title_list)
                            section_title_list = list(section_title_list.split()) 
                            section_title_list = " ".join(section_title_list)
                            title_list.append(section_title_list)
                            print(section_title_list)

                            total_para = []
                            for paragraph in section1.findall('{http://www.elsevier.com/xml/common/dtd}para'):
                                if paragraph != None:
                                    paragraph1 = []
                                    for p in paragraph.itertext():
                                        paragraph1.append(p)

                                    paragraph1 = "".join(paragraph1)
                                    paragraph1 = list(paragraph1.split()) 
                                    paragraph1 = " ".join(paragraph1)
                                    total_para.append(paragraph1)

                            total_para = " ".join(total_para)
                            #print(total_para)
                            para_list.append(total_para)
                            print('\n')

                            for section2 in section1.findall('{http://www.elsevier.com/xml/common/dtd}section'):
                                if section2.find('{http://www.elsevier.com/xml/common/dtd}label') !=None:

                                    label = section2.find('{http://www.elsevier.com/xml/common/dtd}label').text

                                    label_list.append(label)
                                    print('sub-section ', label)

                                    sub_section_title_list = []

                                    sub_section_title = section2.find('{http://www.elsevier.com/xml/common/dtd}section-title')
                                    for sub in sub_section_title.itertext():
                                        sub_section_title_list.append(sub)

                                    sub_section_title_list = "".join(sub_section_title_list)
                                    sub_section_title_list = list(sub_section_title_list.split()) 
                                    sub_section_title_list = " ".join(sub_section_title_list)
                                    title_list.append(sub_section_title_list)
                                    #print('sub-section-title: ', sub_section_title_list)

                                    para1 = []
                                    for para in section2.itertext():
                                    #findall('{http://www.elsevier.com/xml/common/dtd}para'):
                                        #paragraph = para.
                                        #for i in paragraph: 
                                        para1.append(para)

                                    para1 = "".join(para1)
                                    para1 = list(para1.split()) 
                                    para1 = " ".join(para1)
                                    #print(para1) 
                                    para_list.append(para1)
                                    print('\n')

    temp = dict(doi = np.array(doi), 
         title = np.array(title), 
         pub_name = np.array(pub_name), 
         Type = np.array(type_), 
         abstract= np.array(description), 
         authors= np.array(author_list), 
         keyword =np.array(keyword_list),
         title_label = np.array(label_list),
         titles = np.array(title_list),
         text = np.array(para_list))
    
    #print(len(temp['text']), type(temp['text']))
    #print(len(temp['titles']), type(temp['titles']))
    
    df_temp = pd.DataFrame({'sections' : temp['titles'].tolist(), 'Text': temp['text'].tolist()})
    
    #df_temp.head()
    
    #Save each data frame
    df_temp.to_csv(r'{}.csv'.format(i))

### Extract the abstract

In [ ]:
%%time
list_abstract = []
list_doi= []
list_title = []

## ScienceDirect (full-text) document example using DOI
#for i in range(len(data)):
from pprint import pprint
doi= df['DOI']
for i in range(df.shape[0]):
    pprint(doi[i])
    doi_doc = FullDoc(doi = doi[i])
    if doi_doc.read(client):

        #pprint((doi_doc.title))
        abstract = doi_doc._data['coredata']['dc:description']
        title = doi_doc.title
        #pprint(abstract)
        print(i, 'done')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = abstract
        list_abstract.append(abstract)
        list_doi.append(doi[i])
        list_title.append(title)
        #doi_doc.write()

    else:
        pprint('Operation failed')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = 'NAN'
        list_abstract.append('0')
        list_doi.append('1')
        list_title.append('2')

In [ ]:
df_available_abstract = pd.DataFrame()
df_available_abstract['DOI'] = list_doi
df_available_abstract['Title'] = list_title
df_available_abstract['Abstract'] = list_abstract

df_available_abstract.head(10)

In [ ]:
df_available_abstract.to_csv(r'C:\Users\Admin\OneDrive - IIT Delhi\CPCB\PROF. HARI\Jupyter_file\Ex_scibert\Abstract\final_abstract.csv')